In [1]:
#%load_ext nb_mypy

In [2]:
import sys, os

import pybullet as p
import edf_env
from edf_env.env import UR5Env
from edf_env.pybullet_pc_utils import pb_cams_to_pc
from edf_env.pc_utils import pcd_from_numpy, draw_geometry
from edf_env.utils import pb_draw_axis

import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

pybullet build time: Jan 28 2022 20:17:22


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
env = UR5Env(use_gui=True)

In [4]:
# env.disable_gripper_self_collision()

In [5]:
for k,v in env.robot_joint_dict.items():
    if type(k) is int:
        if k > 26:
            print(k, v, env.robot_joint_type_dict[k])

27 EE_joint JOINT_FIXED
28 finger_joint JOINT_REVOLUTE
29 left_outer_finger_joint JOINT_FIXED
30 left_inner_knuckle_joint JOINT_REVOLUTE
31 left_inner_finger_joint JOINT_REVOLUTE
32 right_inner_knuckle_joint JOINT_REVOLUTE
33 right_inner_finger_joint JOINT_REVOLUTE
34 right_outer_knuckle_joint JOINT_REVOLUTE
35 right_outer_finger_joint JOINT_FIXED
36 ur_arm_base_link-base_fixed_joint JOINT_FIXED


In [6]:
# c = p.createConstraint(env.robot_id,
#                        env.robot_joint_dict['left_outer_finger_joint'],
#                        env.robot_id,
#                        env.robot_joint_dict['left_inner_finger_joint'],
#                        jointType=p.JOINT_POINT2POINT,
#                        jointAxis=[0, 0, 0],
#                        parentFramePosition=[0.001, 0, 0.0265],
#                        childFramePosition=[0.012, 0.0, -0.013])
# p.changeConstraint(c, erp=0.1, maxForce=1000)

c = p.createConstraint(parentBodyUniqueId = env.robot_id,
                       parentLinkIndex = env.robot_joint_dict['left_inner_knuckle_joint'],
                       childBodyUniqueId = env.robot_id,
                       childLinkIndex = env.robot_joint_dict['finger_joint'],
                       jointType=p.JOINT_GEAR,
                       jointAxis=[0, 1, 0],
                       parentFramePosition=[0, 0, 0],
                       childFramePosition=[0, 0, 0])
p.changeConstraint(c, gearRatio=-1, maxForce=100,erp=2)

c = p.createConstraint(parentBodyUniqueId = env.robot_id,
                       parentLinkIndex = env.robot_joint_dict['left_inner_knuckle_joint'],
                       childBodyUniqueId = env.robot_id,
                       childLinkIndex = env.robot_joint_dict['left_inner_finger_joint'],
                       jointType=p.JOINT_GEAR,
                       jointAxis=[0, 1, 0],
                       parentFramePosition=[0, 0, 0],
                       childFramePosition=[0, 0, 0])
p.changeConstraint(c, gearRatio=1, maxForce=100,erp=2)

# c = p.createConstraint(parentBodyUniqueId = env.robot_id,
#                        parentLinkIndex = env.robot_joint_dict['finger_joint'],
#                        childBodyUniqueId = env.robot_id,
#                        childLinkIndex = env.robot_joint_dict['left_inner_knuckle_joint'],
#                        jointType=p.JOINT_GEAR,
#                        jointAxis=[0, 1, 0],
#                        parentFramePosition=[0, 0, 0],
#                        childFramePosition=[0, 0, 0])
# p.changeConstraint(c, gearRatio=-1, maxForce=100,erp=2)


# c = p.createConstraint(env.robot_id,
#                        env.robot_joint_dict['right_outer_finger_joint'],
#                        env.robot_id,
#                        env.robot_joint_dict['right_inner_finger_joint'],
#                        jointType=p.JOINT_POINT2POINT,
#                        jointAxis=[0, 0, 0],
#                        parentFramePosition=[-0.001, 0, 0.0265],
#                        childFramePosition=[-0.012, 0.0, -0.013])
# p.changeConstraint(c, erp=0.1, maxForce=1000)

c = p.createConstraint(parentBodyUniqueId = env.robot_id,
                       parentLinkIndex = env.robot_joint_dict['right_inner_knuckle_joint'],
                       childBodyUniqueId = env.robot_id,
                       childLinkIndex = env.robot_joint_dict['right_outer_knuckle_joint'],
                       jointType=p.JOINT_GEAR,
                       jointAxis=[0, -1, 0],
                       parentFramePosition=[0, 0, 0],
                       childFramePosition=[0, 0, 0])
p.changeConstraint(c, gearRatio=-1, maxForce=100,erp=2)

c = p.createConstraint(parentBodyUniqueId = env.robot_id,
                       parentLinkIndex = env.robot_joint_dict['right_outer_knuckle_joint'],
                       childBodyUniqueId = env.robot_id,
                       childLinkIndex = env.robot_joint_dict['right_inner_knuckle_joint'],
                       jointType=p.JOINT_GEAR,
                       jointAxis=[0, -1, 0],
                       parentFramePosition=[0, 0, 0],
                       childFramePosition=[0, 0, 0])
p.changeConstraint(c, gearRatio=-1, maxForce=100,erp=2)



In [7]:
# pb_draw_axis(np.array([0.001, 0, 0.0265]), bodyId=env.robot_id, linkId=env.robot_joint_dict['left_outer_finger_joint'])
# pb_draw_axis(np.array([0.012, 0.0, -0.013]), bodyId=env.robot_id, linkId=env.robot_joint_dict['left_inner_finger_joint'])
# p.removeAllUserDebugItems()

In [8]:
# # gripper constraints
# c = p.createConstraint(env.robot_id, env.robot_joint_dict['left_outer_finger_joint'], env.robot_id, env.robot_joint_dict['left_inner_finger_joint'], p.JOINT_POINT2POINT, [0, 0, 0], [0, -0.014, 0.043], [0, -0.034, 0.021])
# p.changeConstraint(c, erp=0.1, maxForce=1000)

In [9]:
import time

In [13]:
p.setJointMotorControlArray(bodyUniqueId = env.robot_id,
                                            jointIndices = [env.robot_joint_dict['left_inner_knuckle_joint'], env.robot_joint_dict['right_inner_knuckle_joint']],
                                            controlMode = p.POSITION_CONTROL,
                                            targetPositions = [0.0, 0.0])

for _ in range(300):
    #env.gripper_mimic_constraint()
    p.stepSimulation()
    time.sleep(0.003)

In [14]:
p.setJointMotorControlArray(bodyUniqueId = env.robot_id,
                                            jointIndices = [env.robot_joint_dict['left_inner_knuckle_joint'], env.robot_joint_dict['right_inner_knuckle_joint']],
                                            controlMode = p.POSITION_CONTROL,
                                            targetPositions = [0.6, 0])
for _ in range(300):
    #env.gripper_mimic_constraint()
    p.stepSimulation()
    time.sleep(0.003)

In [12]:
fsda

NameError: name 'fsda' is not defined

In [ ]:
pc_coord, pc_color, pc_seg, cam_data_list = env.observe_scene()

In [ ]:
pcd = pcd_from_numpy(coord=pc_coord, color=pc_color, voxel_filter_size = 0.003)

In [ ]:
draw_geometry(pcd)